In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/GoogleDrive-jjluo@terpmail.umd.edu/.shortcut-targets-by-id/1qNzMmGHCg5Xa63Vw3aKbZdMXvkfT2CgC/MedStar/MS_CODE/FieldNN


# Merger

## Module

In [2]:
import torch
from fieldnn.nn.op import MergerLayer
# from ..nn.op import MergerLayer

class Merger_Layer(torch.nn.Module):
    def __init__(self, input_fullname, output_fullname, merger_layer_para):
        super(Merger_Layer, self).__init__()
        
        # n * (bs, .., c_inp) --> (bs, ..., n, c_inp)
        # Meta Info
        self.input_fullname = input_fullname
        self.output_fullname = output_fullname
        
        # Part 0: sizes
        self.input_size = merger_layer_para['input_size']
        self.output_size = merger_layer_para['output_size']
        
        # Part 1: NN
        nn_name, nn_para = merger_layer_para[input_fullname]
        if nn_name.lower() == 'merger':
            self.Merger = MergerLayer()
        else:
            raise ValueError(f'The NN "{nn_name}" is not available')
        
        # Part 2: PostProcess
        self.postprocess = torch.nn.ModuleDict()
        for method, config in merger_layer_para['postprocess'].items():
            if method == 'activator':
                activator = config
                if activator.lower() == 'relu': 
                    self.postprocess[method] = torch.nn.ReLU()
                elif activator.lower() == 'tanh': 
                    self.postprocess[method] = torch.nn.Tanh()
                elif activator.lower() == 'gelu':
                    self.postprocess[method] = torch.nn.GELU()
            elif method == 'dropout':
                self.postprocess[method] = torch.nn.Dropout(**config)
            elif method == 'layernorm':
                self.postprocess[method] = torch.nn.LayerNorm(self.output_size, **config)
                
    def forward(self, input_fullname, fullname2data):
        
        fld_list = input_fullname.split('^')
        assert len(fld_list) == len(fullname2data)
        # (1) holder
        holder = self.Merger([data['holder'] for fld, data in fullname2data.items()], -1)
        
        # (2) merge data
        info = self.Merger([data['info'] for fld, data in fullname2data.items()], -2)
        
        # (3) post-process
        for name, layer in self.postprocess.items():
            info = layer(info)
        
        # (4) names
        # prefix = ['-'.join(i.split('-')[:-1]) for i in fullname2data][0]
        # assert prefix == self.fieldname
        # fullname_new = prefix + '2GrnRec2FldType'
        
        return self.output_fullname, holder, info

In [3]:
# from fieldnn.sublayer.merger import Merger_Layer

## Config

In [4]:
# def get_merger_para(fullname, nn_name, nn_para, input_size, output_size, postprocess):
#     merger_layer_para = {}
    
#     merger_layer_para[fullname] = nn_name, nn_para
    
#     merger_layer_para['input_size'] = input_size
#     merger_layer_para['output_size'] = output_size

#     merger_layer_para['postprocess'] = postprocess
    
#     return merger_layer_para

In [5]:
from fieldnn.utils.parafn import get_merger_para

## Usage

In [7]:
# B_lenP = 3
# B2P_lnEC = [6, 5, 2] # 
# prefix_layers_num = 2
# vocab_size = 100

# Ignore_PSN_Layers = ['B', 'P']

# fullname = 'B-P-EC:Diag-Rec:DiagV-DiagVdftGrn'
# # fld_name = 'B2St2TkGrn'
# info_idx = get_simulated_tensor_from_fldname(fullname, B_lenP, B2P_lnEC, prefix_layers_num, vocab_size)
# print(info_idx.shape)
# holder = torch.LongTensor(info_idx)

# from pprint import pprint
# # embed_size = 512
# vocab_size = 5001
# process = {
#     'dropout' :{'p':0.5, 'inplace':False},
#     'layernorm': {'eps': 1e-05, "elementwise_affine":True},
# }


# ###########
# nn_name = 'Embedding'
# nn_para = {'input_size': vocab_size}
# postprocess = process
# ###########

# input_fullname = fullname
# output_fullname = input_fullname.replace('Grn', '')

# expander_layer_para = get_expander_para(fullname, nn_name, nn_para, embed_size, 
#                                         Ignore_PSN_Layers, 
#                                         postprocess
#                                        )
# pprint(expander_layer_para)
# Expander = Expander_Layer(input_fullname, output_fullname, expander_layer_para)
# print(Expander)


# info = 'None'
# fullname, holder, info = Expander(fullname, holder, info)
# print(info.shape)
# print(fullname)
# print(holder.shape)




In [8]:
# ###########
# nn_name = 'TFM'
# nn_para = {'num_encoder_layers': 6}
# input_size = embed_size
# output_size = embed_size
# embedprocess = process
# postprocess = process
# ###########
# input_fullname = output_fullname
# output_fullname = input_fullname
# learner_layer_para  = get_learner_para(fullname, 
#                                        nn_name, nn_para, 
#                                        input_size, output_size, 
#                                        Ignore_PSN_Layers, 
#                                        embedprocess, postprocess
#                                       )
# pprint(learner_layer_para)

# Learner = Learner_Layer(input_fullname, output_fullname, learner_layer_para)
# # print(Learner)

# fullname, holder, info = Learner(fullname, holder, info)
# print(info.shape)
# print(fullname)
# print(holder.shape)

In [31]:
# def get_fullname_from_inputs(fullname_list):
#     names = [i.split('-')[-1] for i in fullname_list]
    
#     prefix = ['-'.join(i.split('-')[:-1]) for i in fullname_list][0]
    
#     table_row_indicator = list(set([i.count(':') for i in names]))[0]
    
#     if table_row_indicator == 0:
#         fullname = f'{prefix}-{"&".join(names)}'
#     elif table_row_indicator == 1:
#         table_name = [i.split(':')[0] for i in names][0]
#         columns =  [i.split(':')[-1] for i in names]
#         fullname = f'{prefix}-{table_name}-{"&".join(columns)}'
        
#     return fullname

In [33]:
from fieldnn.utils.parafn import get_fullname_from_inputs

## option 1
# fullnamea = 'B-P-EC:Diag-Rec:DiagV-DiagVdftA'
# fullnameb = 'B-P-EC:Diag-Rec:DiagV-DiagVdftB'

## option 2
fullnamea = 'B-P-EC:Diag-Rec:DiagV-VRec:DiagVdftA'
fullnameb = 'B-P-EC:Diag-Rec:DiagV-VRec:DiagVdftB'

fullname_list = [fullnamea, fullnameb]
fullname2data = {i: {'holder': holder, 'info': info} for i in fullname_list}

input_fullname = '^'.join(fullname_list)
print(input_fullname)
output_fullname = get_fullname_from_inputs(fullname_list)
print(output_fullname)

B-P-EC:Diag-Rec:DiagV-VRec:DiagVdftA^B-P-EC:Diag-Rec:DiagV-VRec:DiagVdftB
B-P-EC:Diag-Rec:DiagV-VRec-DiagVdftA&DiagVdftB


In [34]:
for fn_input, data in fullname2data.items():
    print(fn_input, data['holder'].shape, data['info'].shape)
print(fullname_list)

B-P-EC:Diag-Rec:DiagV-VRec:DiagVdftA torch.Size([3, 6, 5, 2]) torch.Size([3, 6, 5, 2, 512])
B-P-EC:Diag-Rec:DiagV-VRec:DiagVdftB torch.Size([3, 6, 5, 2]) torch.Size([3, 6, 5, 2, 512])
['B-P-EC:Diag-Rec:DiagV-VRec:DiagVdftA', 'B-P-EC:Diag-Rec:DiagV-VRec:DiagVdftB']


In [35]:
# fullname_list # in this case, the two fullnames are not the same: 

In [36]:
#########
nn_name = 'Merger'
nn_para = {}
input_size = embed_size
output_size = embed_size
postprocess = process
#########

merger_layer_para = get_merger_para(input_fullname, nn_name, nn_para, input_size, output_size, postprocess)
merger_layer_para

{'B-P-EC:Diag-Rec:DiagV-VRec:DiagVdftA^B-P-EC:Diag-Rec:DiagV-VRec:DiagVdftB': ('Merger',
  {}),
 'input_size': 512,
 'output_size': 512,
 'postprocess': {'dropout': {'p': 0.5, 'inplace': False},
  'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}}

In [37]:
Merger = Merger_Layer(input_fullname, output_fullname, merger_layer_para)
Merger

Merger_Layer(
  (Merger): MergerLayer()
  (postprocess): ModuleDict(
    (dropout): Dropout(p=0.5, inplace=False)
    (layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
)

In [38]:
output_fullname, holder, info = Merger(input_fullname, fullname2data)
print(holder.shape)
print(info.shape)
print(output_fullname)

torch.Size([3, 6, 5, 2, 2])
torch.Size([3, 6, 5, 2, 2, 512])
B-P-EC:Diag-Rec:DiagV-VRec-DiagVdftA&DiagVdftB


In [39]:
info

tensor([[[[[[ 1.1960e-02,  1.1960e-02,  4.4601e-01,  ...,  1.1960e-02,
             -5.3202e-03,  1.1960e-02],
            [-7.0848e-03, -7.0848e-03, -7.0848e-03,  ..., -7.0848e-03,
             -7.0848e-03, -7.0848e-03]],

           [[ 6.5475e-02,  6.5475e-02,  6.0348e-02,  ...,  6.0348e-02,
              6.0348e-02,  7.3955e-02],
            [-6.5196e-02, -6.0204e-02, -6.5196e-02,  ..., -6.5196e-02,
             -6.5196e-02, -6.5196e-02]]],


          [[[ 1.4748e-02,  1.4748e-02,  1.4748e-02,  ..., -4.3735e-02,
             -4.3735e-02, -4.3735e-02],
            [ 2.6660e-02,  6.4197e-03,  7.2832e-03,  ...,  6.4197e-03,
              7.1186e-02,  6.4197e-03]],

           [[-2.5645e-02, -5.3555e-01, -1.0653e+00,  ...,  2.3217e+00,
              5.5946e-02,  2.2144e-01],
            [ 2.6404e-01,  9.9920e-03, -1.0590e+00,  ...,  2.4235e+00,
              9.9920e-03,  2.6404e-01]]],


          [[[-2.3705e-02, -5.6191e-02,  5.1534e-01,  ..., -2.3705e-02,
             -2.3705e-02, -2.

In [40]:
holder

tensor([[[[[20, 20],
           [18, 18]],

          [[ 1,  1],
           [24, 24]],

          [[30, 30],
           [ 0,  0]],

          [[95, 95],
           [88, 88]],

          [[10, 10],
           [ 0,  0]]],


         [[[ 3,  3],
           [87, 87]],

          [[97, 97],
           [ 0,  0]],

          [[37, 37],
           [ 0,  0]],

          [[ 0,  0],
           [ 0,  0]],

          [[ 0,  0],
           [ 0,  0]]],


         [[[10, 10],
           [92, 92]],

          [[43, 43],
           [ 0,  0]],

          [[88, 88],
           [ 0,  0]],

          [[ 0,  0],
           [ 0,  0]],

          [[ 0,  0],
           [ 0,  0]]],


         [[[41, 41],
           [ 0,  0]],

          [[38, 38],
           [68, 68]],

          [[10, 10],
           [55, 55]],

          [[ 0,  0],
           [ 0,  0]],

          [[ 0,  0],
           [ 0,  0]]],


         [[[45, 45],
           [ 0,  0]],

          [[94, 94],
           [93, 93]],

          [[92, 92],
   

In [41]:
print(output_fullname)

B-P-EC:Diag-Rec:DiagV-VRec-DiagVdftA&DiagVdftB


In [42]:
###########
nn_name = 'TFM'
nn_para = {'num_encoder_layers': 6}
input_size = embed_size
output_size = embed_size
embedprocess = process
postprocess = process
###########


input_fullname = output_fullname
output_fullname = input_fullname
learner_layer_para  = get_learner_para(input_fullname, 
                                       nn_name, nn_para, 
                                       input_size, output_size, 
                                       Ignore_PSN_Layers, 
                                       embedprocess, postprocess
                                      )
pprint(learner_layer_para)

Learner = Learner_Layer(input_fullname, output_fullname, learner_layer_para)
# print(Learner)

fullname, holder, info = Learner(input_fullname, holder, info)
print(info.shape)
print(fullname)
print(holder.shape)

{'B-P-EC:Diag-Rec:DiagV-VRec-DiagVdftA&DiagVdftB': ('TFM',
                                                    {'dim_feedforward': 2048,
                                                     'input_size': 512,
                                                     'nhead': 8,
                                                     'num_decoder_layers': 0,
                                                     'num_encoder_layers': 6,
                                                     'output_size': 512,
                                                     'tfm_activation': 'relu',
                                                     'tfm_dropout': 0.1}),
 'Ignore_PSN_Layers': ['B', 'P'],
 'embedprocess': {'dropout': {'inplace': False, 'p': 0.5},
                  'layernorm': {'elementwise_affine': True, 'eps': 1e-05}},
 'input_size': 512,
 'output_size': 512,
 'postprocess': {'dropout': {'inplace': False, 'p': 0.5},
                 'layernorm': {'elementwise_affine': True, 'eps': 1e-05}},
